# RSI Mean Reversion Strategy - Backtesting Analysis

This notebook demonstrates how to backtest the RSI Mean Reversion trading strategy on historical forex data.

## Strategy Overview

The RSI Mean Reversion strategy identifies oversold/overbought conditions and trades mean reversion opportunities.

### Entry Rules
- **LONG**: H1 price > EMA(200) AND M15 price touches lower BB AND RSI < 20
- **SHORT**: H1 price < EMA(200) AND M15 price touches upper BB AND RSI > 80

### Exit Rules
- **Stop Loss**: Beyond BB extreme (1.1× BB distance)
- **Take Profit**: 2× stop loss distance (1:2 R:R)

### Expected Performance
- Win Rate: 55-65%
- Risk/Reward: 1:2
- Trades per day: 2-4
- Best pairs: EUR_USD, GBP_USD

### Files
- Strategy implementation: `src/strategies/rsi_mean_reversion.py`
- Backtest script: `src/backtest_rsi.py`

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest
from pathlib import Path

# Import strategy classes
from src.strategies.rsi_mean_reversion import RSIMeanReversion, RSIMeanReversionOptimized

# Import data fetching classes (for auto-download)
from src.oanda_client import OandaClient
from src.data_retriever import HistoricalDataRetriever
from src.data_storage import DataStorage

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Imports successful")

## 2. Configuration

Set the parameters for our backtest.

In [ ]:
# Data parameters
INSTRUMENT = 'EUR_USD'
GRANULARITY = 'M15'
FROM_DATE = '20240101'
TO_DATE = '20241231'

# Backtest parameters
INITIAL_CASH = 10000
COMMISSION = 0.0001  # 1 pip for forex

# File paths
DATA_DIR = Path('../data/historical')
DATA_PATH = DATA_DIR / INSTRUMENT / f'{INSTRUMENT}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'

# API configuration path
CONFIG_PATH = Path('../config/oanda_config.ini')

print(f"Configuration:")
print(f"  Instrument: {INSTRUMENT}")
print(f"  Timeframe: {GRANULARITY}")
print(f"  Period: {FROM_DATE} to {TO_DATE}")
print(f"  Initial Capital: ${INITIAL_CASH:,.2f}")
print(f"  Commission: {COMMISSION*100:.2f}%")
print(f"  Data Path: {DATA_PATH}")
print(f"  Data Exists: {DATA_PATH.exists()}")

## 3. Load Historical Data

Load the M15 EUR_USD data from our historical data storage.

In [ ]:
def fetch_and_save_data(instrument: str, granularity: str, from_date: str, to_date: str) -> pd.DataFrame:
    """
    Fetch historical data from OANDA API and save to CSV.
    
    Args:
        instrument: Currency pair (e.g., 'EUR_USD')
        granularity: Timeframe (e.g., 'M15')
        from_date: Start date in YYYYMMDD format
        to_date: End date in YYYYMMDD format
        
    Returns:
        DataFrame with OHLCV data
    """
    print(f"Fetching {instrument} {granularity} data from OANDA API...")
    print(f"Date range: {from_date} to {to_date}")
    
    # Convert YYYYMMDD to YYYY-MM-DD for API
    from_date_api = f"{from_date[:4]}-{from_date[4:6]}-{from_date[6:8]}"
    to_date_api = f"{to_date[:4]}-{to_date[4:6]}-{to_date[6:8]}"
    
    # Initialize OANDA client
    client = OandaClient(environment='practice', config_path=str(CONFIG_PATH))
    print("✓ OANDA client initialized")
    
    # Initialize data retriever
    retriever = HistoricalDataRetriever(client)
    
    # Fetch data
    df = retriever.fetch_historical_data(
        instrument=instrument,
        granularity=granularity,
        from_date=from_date_api,
        to_date=to_date_api
    )
    
    if df.empty:
        raise ValueError(f"No data retrieved for {instrument}")
    
    print(f"✓ Retrieved {len(df):,} candles")
    
    # Save to CSV
    storage = DataStorage(base_path=str(DATA_DIR))
    file_path = storage.save_to_csv(
        df=df,
        instrument=instrument,
        granularity=granularity,
        from_date=from_date_api,
        to_date=to_date_api
    )
    print(f"✓ Saved to: {file_path}")
    
    return df


def load_data(data_path: Path) -> pd.DataFrame:
    """
    Load data from CSV file.
    
    Args:
        data_path: Path to CSV file
        
    Returns:
        DataFrame formatted for backtesting.py
    """
    print(f"Loading data from: {data_path}")
    
    # Read CSV, skipping header comments (lines starting with #)
    df = pd.read_csv(data_path, comment='#', parse_dates=['time'], index_col='time')
    
    # Rename columns to match backtesting.py expectations (capitalized)
    df.columns = [col.capitalize() for col in df.columns]
    
    # Keep only required columns
    required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = df[[col for col in required_cols if col in df.columns]]
    
    return df


# Main data loading logic
print("=" * 60)
print("LOADING HISTORICAL DATA")
print("=" * 60)

if DATA_PATH.exists():
    print(f"✓ Data file found: {DATA_PATH}")
    df = load_data(DATA_PATH)
else:
    print(f"✗ Data file not found: {DATA_PATH}")
    print(f"  Downloading from OANDA API...")
    print()
    
    # Fetch and save data
    raw_df = fetch_and_save_data(INSTRUMENT, GRANULARITY, FROM_DATE, TO_DATE)
    
    # Load the saved file (to ensure consistent formatting)
    df = load_data(DATA_PATH)

print(f"\n✓ Loaded {len(df):,} candles")
print(f"  Date range: {df.index.min()} to {df.index.max()}")
print(f"  Duration: {(df.index.max() - df.index.min()).days} days")
print(f"\nData sample:")
display(df.head())
print("\nPrice statistics:")
display(df.describe())

### Visualize Price Data

In [ ]:
# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name=INSTRUMENT
)])

fig.update_layout(
    title=f'{INSTRUMENT} {GRANULARITY} - {FROM_DATE[:4]}',
    yaxis_title='Price',
    xaxis_title='Date',
    height=600,
    xaxis_rangeslider_visible=False
)

fig.show()

## 4. Run Basic Backtest (Base Strategy)

Run the base RSI Mean Reversion strategy with default parameters.

In [ ]:
# Initialize backtest
bt = Backtest(df, RSIMeanReversion, cash=INITIAL_CASH, commission=COMMISSION)

# Run backtest
print("Running backtest with base strategy...\n")
stats = bt.run()

print("=" * 80)
print("BACKTEST RESULTS - BASE STRATEGY")
print("=" * 80)
print(stats)
print("=" * 80)

### Key Metrics Explanation

- **Return [%]**: Total percentage return over the period
- **Sharpe Ratio**: Risk-adjusted return (> 1.0 is good, > 2.0 is excellent)
- **Max Drawdown [%]**: Largest peak-to-trough decline
- **Win Rate [%]**: Percentage of profitable trades
- **Profit Factor**: Gross profit / Gross loss (> 1.0 is profitable)
- **Exposure Time [%]**: Percentage of time in position

## 5. Equity Curve Visualization

Visualize the equity curve and drawdown periods.

In [ ]:
# Extract equity curve
equity = stats['_equity_curve']

# Create subplots
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('Equity Curve', 'Drawdown'),
    row_heights=[0.7, 0.3]
)

# Equity curve
fig.add_trace(
    go.Scatter(x=equity.index, y=equity['Equity'], name='Equity', line=dict(color='blue', width=2)),
    row=1, col=1
)

# Add starting capital line
fig.add_hline(y=INITIAL_CASH, line_dash="dash", line_color="gray", annotation_text="Initial Capital", row=1, col=1)

# Drawdown
fig.add_trace(
    go.Scatter(x=equity.index, y=equity['DrawdownPct'], name='Drawdown',
               fill='tozeroy', line=dict(color='red', width=1)),
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Equity ($)", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)

fig.update_layout(height=800, title_text="Strategy Performance - Base RSI Mean Reversion", showlegend=True)
fig.show()

## 6. Trade Analysis

Analyze individual trades and their characteristics.

In [ ]:
# Extract trades
trades = stats['_trades']

if len(trades) > 0:
    print(f"Total Trades: {len(trades)}")
    print(f"\nTrade Summary:")
    display(trades)

    # Calculate additional metrics
    winning_trades = trades[trades['PnL'] > 0]
    losing_trades = trades[trades['PnL'] < 0]

    print(f"\n=" * 60)
    print("TRADE BREAKDOWN")
    print("=" * 60)
    print(f"Winning trades: {len(winning_trades)} ({len(winning_trades)/len(trades)*100:.1f}%)")
    print(f"Losing trades: {len(losing_trades)} ({len(losing_trades)/len(trades)*100:.1f}%)")
    print(f"\nLong trades: {len(trades[trades['Size'] > 0])} ({len(trades[trades['Size'] > 0])/len(trades)*100:.1f}%)")
    print(f"Short trades: {len(trades[trades['Size'] < 0])} ({len(trades[trades['Size'] < 0])/len(trades)*100:.1f}%)")

    if len(winning_trades) > 0:
        print(f"\nAverage winning trade: ${winning_trades['PnL'].mean():.2f}")
        print(f"Best trade: ${winning_trades['PnL'].max():.2f}")

    if len(losing_trades) > 0:
        print(f"\nAverage losing trade: ${losing_trades['PnL'].mean():.2f}")
        print(f"Worst trade: ${losing_trades['PnL'].min():.2f}")
else:
    print("⚠️ No trades executed. Strategy conditions may be too strict.")

### Trade Distribution

In [ ]:
if len(trades) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Trade returns histogram
    axes[0, 0].hist(trades['ReturnPct'] * 100, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=1)
    axes[0, 0].set_xlabel('Return (%)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Trade Return Distribution')
    axes[0, 0].grid(alpha=0.3)

    # Trade P&L histogram
    axes[0, 1].hist(trades['PnL'], bins=20, edgecolor='black', alpha=0.7, color='green')
    axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=1)
    axes[0, 1].set_xlabel('P&L ($)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Trade P&L Distribution')
    axes[0, 1].grid(alpha=0.3)

    # Trade duration
    durations = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds() / 3600  # hours
    axes[1, 0].hist(durations, bins=20, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 0].set_xlabel('Duration (hours)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Trade Duration Distribution')
    axes[1, 0].grid(alpha=0.3)

    # Cumulative P&L
    cumulative_pnl = trades['PnL'].cumsum()
    axes[1, 1].plot(range(len(cumulative_pnl)), cumulative_pnl, linewidth=2, color='darkblue')
    axes[1, 1].axhline(0, color='red', linestyle='--', linewidth=1)
    axes[1, 1].set_xlabel('Trade Number')
    axes[1, 1].set_ylabel('Cumulative P&L ($)')
    axes[1, 1].set_title('Cumulative Profit/Loss')
    axes[1, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No trades to visualize")

## 7. Optimized Strategy Comparison

Compare the base strategy with the optimized variant that includes additional filters.

In [ ]:
# Run optimized strategy
bt_optimized = Backtest(df, RSIMeanReversionOptimized, cash=INITIAL_CASH, commission=COMMISSION)
stats_optimized = bt_optimized.run()

print("=" * 80)
print("BACKTEST RESULTS - OPTIMIZED STRATEGY")
print("=" * 80)
print(stats_optimized)
print("=" * 80)

### Side-by-Side Comparison

In [ ]:
# Create comparison DataFrame
comparison = pd.DataFrame({
    'Base Strategy': [
        f"{stats['Return [%]']:.2f}%",
        f"{stats['Sharpe Ratio']:.2f}",
        f"{stats['Max. Drawdown [%]']:.2f}%",
        f"{stats['Win Rate [%]']:.2f}%",
        int(stats['# Trades']),
        f"{stats['Exposure Time [%]']:.2f}%",
        f"${stats['Equity Final [$]']:.2f}"
    ],
    'Optimized Strategy': [
        f"{stats_optimized['Return [%]']:.2f}%",
        f"{stats_optimized['Sharpe Ratio']:.2f}",
        f"{stats_optimized['Max. Drawdown [%]']:.2f}%",
        f"{stats_optimized['Win Rate [%]']:.2f}%",
        int(stats_optimized['# Trades']),
        f"{stats_optimized['Exposure Time [%]']:.2f}%",
        f"${stats_optimized['Equity Final [$]']:.2f}"
    ]
}, index=['Return', 'Sharpe Ratio', 'Max Drawdown', 'Win Rate', '# Trades', 'Exposure Time', 'Final Equity'])

print("\n" + "=" * 80)
print("STRATEGY COMPARISON")
print("=" * 80)
display(comparison)

# Overlay equity curves
equity_base = stats['_equity_curve']
equity_optimized = stats_optimized['_equity_curve']

fig = go.Figure()
fig.add_trace(go.Scatter(x=equity_base.index, y=equity_base['Equity'], name='Base Strategy', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=equity_optimized.index, y=equity_optimized['Equity'], name='Optimized Strategy', line=dict(color='green', width=2)))
fig.add_hline(y=INITIAL_CASH, line_dash="dash", line_color="gray", annotation_text="Initial Capital")

fig.update_layout(
    title='Strategy Comparison - Equity Curves',
    xaxis_title='Date',
    yaxis_title='Equity ($)',
    height=600,
    hovermode='x unified'
)
fig.show()

## 8. Parameter Optimization

Find optimal parameter combinations through grid search optimization.

**Note**: This may take 2-5 minutes depending on parameter ranges.

In [ ]:
print("Running parameter optimization...")
print("This may take a few minutes...\n")

# Define optimization ranges
stats_opt = bt.optimize(
    rsi_period=range(10, 21, 2),
    rsi_oversold=range(15, 31, 5),
    rsi_overbought=range(70, 86, 5),
    bb_period=range(15, 26, 5),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.rsi_oversold < 50 and p.rsi_overbought > 50,
    return_heatmap=False
)

print("=" * 80)
print("OPTIMIZATION RESULTS")
print("=" * 80)
print(stats_opt)
print("=" * 80)

### Optimal Parameters

In [ ]:
# Extract optimal parameters
print("\n" + "=" * 60)
print("OPTIMAL PARAMETERS")
print("=" * 60)
print(f"RSI Period: {stats_opt._strategy.rsi_period}")
print(f"RSI Oversold: {stats_opt._strategy.rsi_oversold}")
print(f"RSI Overbought: {stats_opt._strategy.rsi_overbought}")
print(f"BB Period: {stats_opt._strategy.bb_period}")
print(f"BB Std Dev: {stats_opt._strategy.bb_std}")
print(f"\nOptimized Sharpe Ratio: {stats_opt['Sharpe Ratio']:.2f}")
print(f"Optimized Return: {stats_opt['Return [%]']:.2f}%")
print(f"Optimized Win Rate: {stats_opt['Win Rate [%]']:.2f}%")

## 9. Conclusions

### Key Findings

1. **Strategy Performance**: The RSI Mean Reversion strategy shows [analyze results here]
2. **Trade Frequency**: Strategy generates [number] trades over the year
3. **Win Rate**: Actual win rate is [X]%, [compared to expected 55-65%]
4. **Risk Management**: Max drawdown of [X]% shows [assessment]
5. **Optimization**: Parameter optimization [improved/didn't improve] performance significantly

### Strategy Strengths
- Clear entry/exit rules based on technical indicators
- Defined risk management with stop loss and take profit
- Trend filter (EMA200) helps avoid counter-trend trades
- Mean reversion edge in ranging markets

### Strategy Weaknesses
- May generate few signals if conditions are too strict
- Sensitive to parameter selection
- Performance depends on market regime (range-bound vs trending)
- Requires sufficient volatility for mean reversion

### Recommendations
1. **Parameter Tuning**: Consider using optimized parameters for live trading
2. **Walk-Forward Analysis**: Test on out-of-sample data to avoid overfitting
3. **Market Regime Filter**: Add volatility filter to avoid low-volatility periods
4. **Position Sizing**: Implement dynamic position sizing based on account equity
5. **Risk Management**: Consider reducing risk_reward ratio if win rate is high

## 10. Next Steps

Now that you've analyzed the strategy performance, here are suggested next steps:

### 1. Test on Different Instruments
- Retrieve data for GBP_USD, USD_JPY, AUD_USD
- Run same backtest to see if strategy is instrument-specific
- Compare performance across major pairs

### 2. Walk-Forward Analysis
- Split data into in-sample (training) and out-of-sample (testing)
- Optimize on in-sample, validate on out-of-sample
- Check for overfitting

### 3. Different Timeframes
- Test on H1 (1-hour) and H4 (4-hour) data
- Compare trade frequency and performance
- Find optimal timeframe for your trading style

### 4. Add Position Sizing
- Implement Kelly Criterion or fixed fractional position sizing
- Risk 1-2% of capital per trade
- See `src/utils/position_sizing.py` (to be created)

### 5. Paper Trading
- Test strategy on OANDA practice account
- Monitor real-time performance
- Adjust parameters based on live market conditions

### 6. Create Trading Journal
- Log all trades with entry/exit reasons
- Track emotions and decision-making process
- Review weekly/monthly to improve

### 7. Strategy Enhancements
- Add volatility filter (ATR, BB width)
- Implement time-of-day filter (London/NY session)
- Add news event filter
- Consider machine learning for signal confirmation

---

**Remember**: Past performance does not guarantee future results. Always paper trade before going live!